In [11]:
%%capture
pip install overpy

In [6]:
import pandas as pd

file_path1 = 'competitors_MI.csv'
competitors_MI = pd.read_csv(file_path1)

file_path2 = 'meijer_stores_MI.csv'
meijer_stores_MI = pd.read_csv(file_path2)

In [7]:
meijer_stores_MI.head(4)

name                                            address   latitude  \
0  Meijer         1251 M-32, Alpena, MI 49707, United States  45.059147   
1  Meijer    100 Pigeon Rd, Bad Axe, MI 48413, United States  43.825699   
2  Meijer  9515 Birch Run Rd, Birch Run, MI 48415, United...  43.252918   
3  Meijer  3360 Tittabawassee Rd, Saginaw, MI 48604, Unit...  43.482030   

   longitude  county  
0 -83.474693  Alcona  
1 -83.004745  Alcona  
2 -83.765800  Alcona  
3 -83.982479  Alcona

In [38]:
meijer_store_MI_sample = meijer_stores_MI.head(4)

In [8]:
competitors_MI.head(4)

name                                            address   latitude  \
0  Kum & Go  4800 44th St SW, Grandville, MI 49418, United ...  42.884595   
1  Kum & Go  5437 Division Ave S, Wyoming, MI 49548, United...  42.865430   
2  Kum & Go  4560 Chicago Dr SW, Grandville, MI 49418, Unit...  42.905083   
3  Kum & Go  2134 Alpine Ave NW, Walker, MI 49544, United S...  43.002258   

   longitude       County  
0 -85.781344  Kent County  
1 -85.664682  Kent County  
2 -85.776313  Kent County  
3 -85.688179  Kent County

In [39]:
competitors_MI_sample = competitors_MI.head(4)

In [40]:
import overpy
import pandas as pd
import time

# Initialize Overpass API
api = overpy.Overpass()

# Convert miles to meters for the radius
radius_miles = 1
radius = radius_miles * 1609.34

# Define categories and their tags
categories_tags = {
    
    "Grocery Stores": [
        ("amenity", "supermarket"),
        ("shop", "Bakery"),
        ("shop", "Convenience"),
        ("shop", "Frozen Food"),
        ("shop", "Department Store"),
        ("shop", "Supermarket"),
        ("shop", "Wholesale"),
    ],
    
    "Food Outlets": [
        ("amenity", "Fast Food"),
        ("amenity", "Food Court"),
        ("amenity", "restaurant"),
        ("amenity", "Ice Cream"),
        ("shop", "Cheese"),
        ("shop", "Chocolate"),
        ("shop", "Ice Cream"),
        ("shop", "Pasta"),
        ("shop", "Pastry"),
        ("shop", "Seafood"),
        ("shop", "Food"),
    ],
    
    "Tourist Destinations": [
        ("tourism", "attraction"),
        ("amenity", "Events Venue"),
        ("amenity", "Exhibition Centre"),
        ("boundary", "national_park"),
        ("boundary", "Museum"),
        ("building", "Beach Hut"),
        ("building", "Castle"),
    ],
    
    "Hotels": [
        ("building", "hotel"),
    ],
    
    "Educational Institutions": [
        ("building", "College"),
        ("building", "School"),
        ("amenity", "library"),
        ("building", "University"),
    ],
    
    "Hospitals": [
        ("amenity", "Clinic"),
        ("amenity", "Nursing Home"),
        ("amenity", "Pharmacy"),
        ("building", "hospital"),
    ],
    
    "Airports": [
        ("aeroway", "helipad"),
        ("aeroway", "heliport"),
        ("aeroway", "runway"),
        ("aeroway", "terminal"),
    ],
    
    "Residential Housing Density": [
        ("building", "bungalow"),
        ("building", "apartments"),
        ("building", "residential"),
        ("building", "Hut"),
    ],
    
    "Commercial Housing Density": [
        ("building", "commercial"),
        ("building", "office"),
    ],
    
    "Fuel Stations & Car Wash": [
        ("amenity", "Car Wash"),
        ("amenity", "Charging Station"),
        ("amenity", "Fuel"),
    ],
    
    "Apparels": [
        ("shop", "Clothes"),
        ("shop", "Fabric"),
        ("shop", "Fashion"),
    ],
    
    "Height Restrictor": [
        ("barrier", "height_restrictor"),
    ],
    
    "Border Zone": [
        ("boundary", "border_zone"),
    ],
    
    "Public Transportation": [
        ("public_transport", "Public Transport Platform"),
        ("public_transport", "Public Transport Station"),
        ("public_transport", "Railway Station"),
        ("public_transport", "Railway Subway Entrance"),
        ("public_transport", "Railway Tram Stop"),
    ],
}


# Define a delay duration in seconds (e.g., 10 seconds)
delay_duration = 10

def fetch_feature_count(api, lat, lon, radius, category_tags):
    """
    Fetch count of features for the given category tags.

    :param api: Overpass API instance.
    :param lat: Latitude of the center.
    :param lon: Longitude of the center.
    :param radius: Radius for the search.
    :param category_tags: Dictionary of categories and their tags.
    :return: Dictionary with counts of features for each category.
    """
    category_counts = {}

    for category, tags in category_tags.items():
        print(f"Fetching data for category: {category}")
        category_count = 0
        for tag_pair in tags:
            category_name, tag_name = tag_pair
            query = f"""
                (node["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 way["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 rel["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                );out;
            """
            result = api.query(query)
            category_count += len(result.nodes) + len(result.ways) + len(result.relations)
        
        category_counts[category] = category_count
        print(f"Completed fetching for category: {category}. Count: {category_count}")
        
        # Delay to manage server load
        print(f"Waiting for {delay_duration} seconds to manage server load.")
        time.sleep(delay_duration)

    return category_counts

def main(input_df):
    all_feature_counts = []

    for index, row in input_df.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        print(f"Processing location at latitude: {lat}, longitude: {lon}")
        feature_counts = fetch_feature_count(api, lat, lon, radius, categories_tags)
        feature_counts.update({
            "latitude": lat,
            "longitude": lon,
            "radius_miles": radius_miles
        })
        all_feature_counts.append(feature_counts)

    # Convert the list of dictionaries to a DataFrame
    meijer_stores_MI = pd.DataFrame(all_feature_counts)

    # Display the DataFrame
    print(meijer_stores_MI)

    # Output the DataFrame to a CSV file
    meijer_stores_MI.to_csv("meijer_nearby_places_final.csv", index=False)
    print("Data collection complete. CSV file created.")

if __name__ == "__main__":
    # Input DataFrame with 'latitude' and 'longitude' columns
    meijer_nearby_places_final = main(meijer_store_MI_sample)

Processing location at latitude: 45.0591468, longitude: -83.4746932
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 10 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 2
Waiting for 10 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 10 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 10 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 0
Waiting for 10 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 10 seconds to manage server load.
Fetching data for category: Airports
Completed f

Completed fetching for category: Apparels. Count: 0
Waiting for 10 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 10 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 10 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 10 seconds to manage server load.
   Grocery Stores  Food Outlets  Tourist Destinations  Hotels  \
0               0             2                     0       0   
1               0             3                     0       0   
2               0             8                     0       1   
3               0            19                     0       0   

   Educational Institutions  Hospitals  Airports  Residential Housing Density  \
0                         0          0  

In [41]:
file_path = 'meijer_nearby_places_final.csv'
nearby_places_final = pd.read_csv(file_path)
nearby_places_final.head()

Grocery Stores  Food Outlets  Tourist Destinations  Hotels  \
0               0             2                     0       0   
1               0             3                     0       0   
2               0             8                     0       1   
3               0            19                     0       0   

   Educational Institutions  Hospitals  Airports  Residential Housing Density  \
0                         0          0         0                           17   
1                         0          0         0                            0   
2                         0          0         0                            0   
3                         0          1         0                            0   

   Commercial Housing Density  Fuel Stations & Car Wash  Apparels  \
0                           5                         0         0   
1                           1                         0         0   
2                           2                         0         0   
3                           6                         0         0   

   Height Restrictor  Border Zone  Public Transportation   latitude  \
0                  0            0                      0  45.059147   
1                  0            0                      0  43.825699   
2                  0            0                      0  43.252918   
3                  0            0                      0  43.482030   

   longitude  radius_miles  
0 -83.474693             1  
1 -83.004745             1  
2 -83.765800             1  
3 -83.982479             1

In [34]:
meijer_stores_MI

name  \
0                        Meijer   
1                        Meijer   
2                        Meijer   
3                        Meijer   
4    Meijer Express Gas Station   
..                          ...   
199  Meijer Express Gas Station   
200  Meijer Express Gas Station   
201  Meijer Express Gas Station   
202                      Meijer   
203  Meijer Express Gas Station   

                                               address   latitude  longitude  \
0           1251 M-32, Alpena, MI 49707, United States  45.059147 -83.474693   
1      100 Pigeon Rd, Bad Axe, MI 48413, United States  43.825699 -83.004745   
2    9515 Birch Run Rd, Birch Run, MI 48415, United...  43.252918 -83.765800   
3    3360 Tittabawassee Rd, Saginaw, MI 48604, Unit...  43.482030 -83.982479   
4           1241 M-32, Alpena, MI 49707, United States  45.061162 -83.474606   
..                                                 ...        ...        ...   
199   521 N Pine Rd, Bay City, MI 48708, United States  43.580492 -83.837882   
200  5671 Jackson Rd, Ann Arbor, MI 48103, United S...  42.287728 -83.842059   
201  3995 Carpenter Rd, Ypsilanti, MI 48197, United...  42.231416 -83.679345   
202  1475 E Jefferson Ave, Detroit, MI 48207, Unite...  42.334396 -83.032600   
203  1231 Eight Mile Rd, Detroit, MI 48203, United ...  42.446126 -83.118155   

        county  
0       Alcona  
1       Alcona  
2       Alcona  
3       Alcona  
4       Alcona  
..         ...  
199    Tuscola  
200  Washtenaw  
201  Washtenaw  
202      Wayne  
203      Wayne  

[204 rows x 5 columns]

In [42]:
final_nearby_places_meijer = pd.merge(meijer_stores_MI, nearby_places_final, on=['latitude','longitude'], how='inner')
final_nearby_places_meijer

name                                            address   latitude  \
0  Meijer         1251 M-32, Alpena, MI 49707, United States  45.059147   
1  Meijer    100 Pigeon Rd, Bad Axe, MI 48413, United States  43.825699   
2  Meijer  9515 Birch Run Rd, Birch Run, MI 48415, United...  43.252918   
3  Meijer  3360 Tittabawassee Rd, Saginaw, MI 48604, Unit...  43.482030   

   longitude  county  Grocery Stores  Food Outlets  Tourist Destinations  \
0 -83.474693  Alcona               0             2                     0   
1 -83.004745  Alcona               0             3                     0   
2 -83.765800  Alcona               0             8                     0   
3 -83.982479  Alcona               0            19                     0   

   Hotels  Educational Institutions  Hospitals  Airports  \
0       0                         0          0         0   
1       0                         0          0         0   
2       1                         0          0         0   
3       0                         0          1         0   

   Residential Housing Density  Commercial Housing Density  \
0                           17                           5   
1                            0                           1   
2                            0                           2   
3                            0                           6   

   Fuel Stations & Car Wash  Apparels  Height Restrictor  Border Zone  \
0                         0         0                  0            0   
1                         0         0                  0            0   
2                         0         0                  0            0   
3                         0         0                  0            0   

   Public Transportation  radius_miles  
0                      0             1  
1                      0             1  
2                      0             1  
3                      0             1

In [43]:
 final_nearby_places_meijer.to_csv("final_nearby_places_meijer.csv", index=False)

# Appendix

In [ ]:
import overpy
import pandas as pd

# Initialize Overpass API
api = overpy.Overpass()

# Convert miles to meters for the radius
radius_miles = 1
radius = radius_miles * 1609.34

# Define categories and their tags
categories_tags = {
    
    "Grocery Stores": [
        ("amenity", "supermarket"),
        ("shop", "Bakery"),
        ("shop", "Convenience"),
        ("shop", "Frozen Food"),
        ("shop", "Department Store"),
        ("shop", "Supermarket"),
        ("shop", "Wholesale"),
    ],
    
    "Food Outlets": [
        ("amenity", "Fast Food"),
        ("amenity", "Food Court"),
        ("amenity", "restaurant"),
        ("amenity", "Ice Cream"),
        ("shop", "Cheese"),
        ("shop", "Chocolate"),
        ("shop", "Ice Cream"),
        ("shop", "Pasta"),
        ("shop", "Pastry"),
        ("shop", "Seafood"),
        ("shop", "Food"),
    ],
    
    "Tourist Destinations": [
        ("tourism", "attraction"),
        ("amenity", "Events Venue"),
        ("amenity", "Exhibition Centre"),
        ("boundary", "national_park"),
        ("boundary", "Museum"),
        ("building", "Beach Hut"),
        ("building", "Castle"),
    ],
    
    "Hotels": [
        ("building", "hotel"),
    ],
    
    "Educational Institutions": [
        ("building", "College"),
        ("building", "School"),
        ("amenity", "library"),
        ("building", "University"),
    ],
    
    "Hospitals": [
        ("amenity", "Clinic"),
        ("amenity", "Nursing Home"),
        ("amenity", "Pharmacy"),
        ("building", "hospital"),
    ],
    
    "Airports": [
        ("aeroway", "helipad"),
        ("aeroway", "heliport"),
        ("aeroway", "runway"),
        ("aeroway", "terminal"),
    ],
    
    "Residential Housing Density": [
        ("building", "bungalow"),
        ("building", "apartments"),
        ("building", "residential"),
        ("building", "Hut"),
    ],
    
    "Commercial Housing Density": [
        ("building", "commercial"),
        ("building", "office"),
    ],
    
    "Fuel Stations & Car Wash": [
        ("amenity", "Car Wash"),
        ("amenity", "Charging Station"),
        ("amenity", "Fuel"),
    ],
    
    "Apparels": [
        ("shop", "Clothes"),
        ("shop", "Fabric"),
        ("shop", "Fashion"),
    ],
    
    "Height Restrictor": [
        ("barrier", "height_restrictor"),
    ],
    
    "Border Zone": [
        ("boundary", "border_zone"),
    ],
    
    "Public Transportation": [
        ("public_transport", "Public Transport Platform"),
        ("public_transport", "Public Transport Station"),
        ("public_transport", "Railway Station"),
        ("public_transport", "Railway Subway Entrance"),
        ("public_transport", "Railway Tram Stop"),
    ],
}


def fetch_feature_count(api, lat, lon, radius, category_tags):
    """
    Fetch count of features for the given category tags.

    :param api: Overpass API instance.
    :param lat: Latitude of the center.
    :param lon: Longitude of the center.
    :param radius: Radius for the search.
    :param category_tags: Dictionary of categories and their tags.
    :return: Dictionary with counts of features for each category.
    """
    category_counts = {}

    for category, tags in category_tags.items():
        category_count = 0
        for tag_pair in tags:
            category_name, tag_name = tag_pair
            query = f"""
                (node["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 way["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 rel["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                );out;
            """
            result = api.query(query)
            category_count += len(result.nodes) + len(result.ways) + len(result.relations)
        
        category_counts[category] = category_count

    return category_counts

def main(input_df):
    all_feature_counts = []

    for index, row in input_df.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        feature_counts = fetch_feature_count(api, lat, lon, radius, categories_tags)
        feature_counts.update({
            "latitude": lat,
            "longitude": lon,
            "radius_miles": radius_miles
        })
        all_feature_counts.append(feature_counts)

    # Convert the list of dictionaries to a DataFrame
    meijer_stores_MI = pd.DataFrame(all_feature_counts)

    # Output the DataFrame to a CSV file
    meijer_stores_MI.to_csv("meijer_nearby_places_final.csv", index=False)

if __name__ == "__main__":
    # Input DataFrame with 'latitude' and 'longitude' columns
    meijer_nearby_places_final = main(meijer_stores_MI_sample)